In [1]:
# Import necessary libraries
import requests
import pandas as pd

# Base URL for PokeAPI
BASE_URL = "https://pokeapi.co/api/v2/"

# Function to fetch data from a URL
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to get data for all Pokémon species and flatten it
def get_all_pokemon_species():
    url = BASE_URL + "pokemon-species/"
    species_data_list = []
    growth_rate_list = []
    pokedex_numbers_list = []
    egg_groups_list = []
    flavor_text_list = []
    genera_list = []
    varieties_list = []

    while url:
        data = fetch_data(url)
        if data:
            for species in data['results']:
                species_info = fetch_data(species['url'])
                if species_info:
                    # Extracting Pokémon species basic data
                    species_data = {
                        'id': species_info['id'],
                        'name': species_info['name'],
                        'order': species_info['order'],
                        'gender_rate': species_info['gender_rate'],
                        'capture_rate': species_info['capture_rate'],
                        'base_happiness': species_info['base_happiness'],
                        'is_baby': species_info['is_baby'],
                        'is_legendary': species_info['is_legendary'],
                        'is_mythical': species_info['is_mythical'],
                        'hatch_counter': species_info['hatch_counter'],
                        'has_gender_differences': species_info['has_gender_differences'],
                        'forms_switchable': species_info['forms_switchable'],
                        'color': species_info['color']['name'] if species_info['color'] else None,
                        'shape': species_info['shape']['name'] if species_info['shape'] else None,
                        'evolves_from_species': species_info['evolves_from_species']['name'] if species_info['evolves_from_species'] else None,
                        'evolution_chain_url': species_info['evolution_chain']['url'] if species_info['evolution_chain'] else None,
                        'habitat': species_info['habitat']['name'] if species_info['habitat'] else None,
                        'generation': species_info['generation']['name'] if species_info['generation'] else None
                    }
                    species_data_list.append(species_data)

                    # Extracting growth rate data
                    if species_info['growth_rate']:
                        growth_rate_list.append({
                            'species_id': species_info['id'],
                            'growth_rate_name': species_info['growth_rate']['name']
                        })

                    # Extracting pokedex numbers data
                    pokedex_numbers = [
                        {
                            'species_id': species_info['id'],
                            'entry_number': pokedex['entry_number'],
                            'pokedex_name': pokedex['pokedex']['name']
                        }
                        for pokedex in species_info['pokedex_numbers']
                    ]
                    pokedex_numbers_list.extend(pokedex_numbers)

                    # Extracting egg groups data
                    egg_groups = [
                        {
                            'species_id': species_info['id'],
                            'egg_group_name': egg_group['name']
                        }
                        for egg_group in species_info['egg_groups']
                    ]
                    egg_groups_list.extend(egg_groups)

                    # Extracting flavor text entries data
                    flavor_text_entries = [
                        {
                            'species_id': species_info['id'],
                            'flavor_text': flavor['flavor_text'],
                            'language': flavor['language']['name'],
                            'version': flavor['version']['name']
                        }
                        for flavor in species_info['flavor_text_entries'] if flavor['language']['name'] == 'en'
                    ]
                    flavor_text_list.extend(flavor_text_entries)

                    # Extracting genera data
                    genera = [
                        {
                            'species_id': species_info['id'],
                            'genus': genus['genus'],
                            'language': genus['language']['name']
                        }
                        for genus in species_info['genera'] if genus['language']['name'] == 'en'
                    ]
                    genera_list.extend(genera)

                    # Extracting varieties data
                    varieties = [
                        {
                            'species_id': species_info['id'],
                            'variety_name': variety['pokemon']['name'],
                            'is_default': variety['is_default']
                        }
                        for variety in species_info['varieties']
                    ]
                    varieties_list.extend(varieties)

            # Update the URL for the next page of results
            url = data['next']
        else:
            break

    # Creating DataFrames for each category of data
    species_df = pd.DataFrame(species_data_list)
    growth_rate_df = pd.DataFrame(growth_rate_list)
    pokedex_numbers_df = pd.DataFrame(pokedex_numbers_list)
    egg_groups_df = pd.DataFrame(egg_groups_list)
    flavor_text_df = pd.DataFrame(flavor_text_list)
    genera_df = pd.DataFrame(genera_list)
    varieties_df = pd.DataFrame(varieties_list)
    
    return species_df, growth_rate_df, pokedex_numbers_df, egg_groups_df, flavor_text_df, genera_df, varieties_df

# Example usage for all Pokémon species
species_df, growth_rate_df, pokedex_numbers_df, egg_groups_df, flavor_text_df, genera_df, varieties_df = get_all_pokemon_species()



In [2]:
# Displaying the DataFrames
species_df


,id,name,order,gender_rate,capture_rate,base_happiness,is_baby,is_legendary,is_mythical,hatch_counter,has_gender_differences,forms_switchable,color,shape,evolves_from_species,evolution_chain_url,habitat,generation
0,1,bulbasaur,1,1,45,50.0,False,False,False,20.0,False,False,green,quadruped,None,https://pokeapi.co/api/v2/evolution-chain/1/,grassland,generation-i
1,2,ivysaur,2,1,45,50.0,False,False,False,20.0,False,False,green,quadruped,bulbasaur,https://pokeapi.co/api/v2/evolution-chain/1/,grassland,generation-i
2,3,venusaur,3,1,45,50.0,False,False,False,20.0,True,True,green,quadruped,ivysaur,https://pokeapi.co/api/v2/evolution-chain/1/,grassland,generation-i
3,4,charmander,4,1,45,50.0,False,False,False,20.0,False,False,red,upright,None,https://pokeapi.co/api/v2/evolution-chain/2/,mountain,generation-i
4,5,charmeleon,5,1,45,50.0,False,False,False,20.0,False,False,red,upright,charmander,https://pokeapi.co/api/v2/evolution-chain/2/,mountain,generation-i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1021,raging-bolt,1023,-1,10,0.0,False,False,False,50.0,False,False,yellow,quadruped,None,https://pokeapi.co/api/v2/evolution-chain/545/,None,generation-ix
1021,1022,iron-boulder,1024,-1,10,0.0,False,False,False,50.0,False,False,gray,quadruped,None,https://pokeapi.co/api/v2/evolution-chain/547/,None,generation-ix
1022,1023,iron-crown,1025,-1,10,0.0,False,False,False,50.0,False,False,blue,quadruped,None,https://pokeapi.co/api/v2/evolution-chain/546/,None,generation-ix
1023,1024,terapagos,1026,-1,255,50.0,False,True,False,5.0,False,False,blue,quadruped,None,https://pokeapi.co/api/v2/evolution-chain/548/,None,generation-ix


In [3]:
growth_rate_df


,species_id,growth_rate_name
0,1,medium-slow
1,2,medium-slow
2,3,medium-slow
3,4,medium-slow
4,5,medium-slow
...,...,...
1020,1021,slow
1021,1022,slow
1022,1023,slow
1023,1024,slow


In [4]:
pokedex_numbers_df


,species_id,entry_number,pokedex_name
0,1,1,national
1,1,1,kanto
2,1,226,original-johto
3,1,231,updated-johto
4,1,80,kalos-central
...,...,...,...
7407,1023,238,blueberry
7408,1024,1024,national
7409,1024,240,blueberry
7410,1025,1025,national


In [5]:
egg_groups_df


,species_id,egg_group_name
0,1,monster
1,1,plant
2,2,monster
3,2,plant
4,3,monster
...,...,...
1279,1006,no-eggs
1280,1007,no-eggs
1281,1008,no-eggs
1282,1009,no-eggs


In [6]:
flavor_text_df


,species_id,flavor_text,language,version
0,1,A strange seed was\nplanted on its\nback at bi...,en,red
1,1,A strange seed was\nplanted on its\nback at bi...,en,blue
2,1,It can go for days\nwithout eating a\nsingle m...,en,yellow
3,1,The seed on its\nback is filled\nwith nutrient...,en,gold
4,1,It carries a seed\non its back right\nfrom bir...,en,silver
...,...,...,...,...
14491,1023,There was supposedly an incident in which it l...,en,violet
14492,1024,Terapagos protects itself using its power to t...,en,scarlet
14493,1024,It’s thought that this Pokémon lived in ancien...,en,violet
14494,1025,It feeds others toxic mochi that draw out desi...,en,scarlet


In [7]:
genera_df


,species_id,genus,language
0,1,Seed Pokémon,en
1,2,Seed Pokémon,en
2,3,Seed Pokémon,en
3,4,Lizard Pokémon,en
4,5,Flame Pokémon,en
...,...,...,...
1020,1021,Paradox Pokémon,en
1021,1022,Paradox Pokémon,en
1022,1023,Paradox Pokémon,en
1023,1024,Tera Pokémon,en


In [8]:
varieties_df


,species_id,variety_name,is_default
0,1,bulbasaur,True
1,2,ivysaur,True
2,3,venusaur,True
3,3,venusaur-mega,False
4,3,venusaur-gmax,False
...,...,...,...
1297,1023,iron-crown,True
1298,1024,terapagos,True
1299,1024,terapagos-terastal,False
1300,1024,terapagos-stellar,False


In [9]:

# Save DataFrames to CSV files
species_df.to_csv('species.csv', index=False)
growth_rate_df.to_csv('growth_rate.csv', index=False)
pokedex_numbers_df.to_csv('pokedex_numbers.csv', index=False)
egg_groups_df.to_csv('egg_groups.csv', index=False)
flavor_text_df.to_csv('flavor_text.csv', index=False)
genera_df.to_csv('genera.csv', index=False)
varieties_df.to_csv('varieties.csv', index=False)
